In [1]:
%load_ext gams_magic

# The shortest path problem: 

Find the shortest path from node s to node y in the following directed network

![alt text](shortestpath.png "")

In [10]:
%%gams

#definitions of sets
set Nodes /s, u, x, z, v, y/;
alias(j,Nodes);
set Arcs(Nodes,Nodes) /s.u, u.s, s.x, u.x, x.u, u.z, z.u, x.z, x.y, z.y, u.v, z.v,
         v.y/;

parameters c(Nodes, Nodes) /
s.u 1, u.s 1, s.x 3, u.x 3, x.u 3, u.z 2, z.u 3, x.z 1, x.y 2, z.y 3, 
u.v 4, z.v 1, v.y 2/;

parameters netsupply(Nodes) /s 1, y -1/;

#definition of variables 
variables x(Nodes, Nodes), totalcost; 

positive variable x;

#definition of equations
equations balance(Nodes), obj;

balance(Nodes).. sum(j$Arcs(Nodes,j), x(Nodes, j)) 
    - sum(j$Arcs(j,Nodes), x(j,Nodes)) =e= netsupply(Nodes); 

obj.. totalcost =e= sum(Arcs, c(Arcs)*x(Arcs));     

model shortestpath/all/;
solve shortestpath using lp minimizing totalcost; 
    
    

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),5.0,7,14,LP,OSICPLEX,0


In [9]:
%gams display x.l, totalcost.l;
%gams_lst -e

E x e c u t i o n


----     36 VARIABLE x.L  

            x           y

s       1.000
x                   1.000


----     36 VARIABLE totalcost.L           =        5.000  




In [ ]:
%%gams

set N "nodes" /s,u,x,v,z,y/;
set A(N,N) "arcs" /s.(u,x),  u.(s,x,z,v),  x.(u,z,y), z.(u,v,y),   v.y/;
alias (N,j);

parameters S(N) "net supplies for nodes" /s 1, y -1/, 
#supplies for other nodes will be set to 0 by default

         c(N,N) "unit cost on arcs" /
          s.u   1
          s.x   3
          u.s   1
          u.x   3
          u.z   2
          u.v   4
          x.u   3
          x.z   1
          x.y   2
          z.u   3
          z.v   1
          z.y   3
          v.y   2/;

variable totalcost, x(N,N);
positive variable x;

equations balance(N), obj;

balance(N)..
        sum(j$A(N,j), x(N,j)) - sum(j$A(j,N), x(j,N)) =e= s(N);
obj..
        totalcost =e= sum(A, c(A)*x(A));

model sp/balance, obj/;

#Add the following statements to inform GAMS to use the network simplex method to solve your model.
#The $ symbol has to be the first character in the line.

option lp = cplex;
$onecho > cplex.opt
lpmethod 3
$offecho
sp.optfile = 1;


#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

solve sp using lp minimizing totalcost;


In [ ]:
%gams_lst

In [ ]:
%gams_lst

In [ ]:
%gams display x.l, totalcost.l;
%gams_lst -e